In [1]:
from NextVisionML import MLContext
from NextVisionML import VarianceFilter, DecisionTreeClfr, OneHotClfr, PcaUnsupervised

In [2]:
#Create an context (Loads data)
mlcontext = MLContext()

In [ ]:
#add feature selection methods
mlcontext.train_preparation_methods.append(VarianceFilter(mlcontext)) 
#Add data prepocessing methods
mlcontext.train_methods.append(OneHotClfr(mlcontext))
mlcontext.train_methods.append(DecisionTreeClfr(mlcontext))
#select a model type
#start the training process
mlcontext.start_train_process()

In [ ]:
import pandas as pd
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split

In [ ]:
import hyperopt

In [ ]:
from NextVisionML import load_context

sqlContext, train_with_labels, test_with_labels = load_context()

In [ ]:
test_with_labels

In [ ]:
train_dataset = pd.concat([mlcontext.iter_train_X[2], mlcontext.iter_train_y[2]], axis = 0)
cols = []#["Amb_WindSpeed_Std", "Avg_Precipitation", "Grd_Prod_CosPhi_Avg", "Max_Precipitation", "Min_Precipitation"]
train_dataset = train_dataset.drop(columns=cols)
d = dice_ml.Data(
    dataframe=train_dataset,
    continuous_features= list(train_dataset.drop(columns=["Risk Level"]).columns),
    outcome_name="Risk Level")

# Using sklearn backend
m = dice_ml.Model(model=mlcontext.iter_objs[2]["model"], backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [ ]:
test_data = mlcontext.iter_test_X[2].drop(columns = cols)

In [ ]:
e1 = exp.generate_counterfactuals(test_data.iloc[:5].copy(), total_CFs=2, desired_class=0)

In [ ]:
e1.visualize_as_dataframe(show_only_changes=False)

In [ ]:
import torch
from sklearn.metrics import balanced_accuracy_score

In [ ]:
class SimpleNetSoftmax(torch.nn.Module):
    def __init__(self, input_size, output_size, layer_count):
        super(SimpleNetSoftmax, self).__init__()
        self.call_super_init
        self.fc = torch.nn.ModuleList()
        self.lc = layer_count
        for _ in range(layer_count-1):
            self.fc.append(torch.nn.Linear(input_size, input_size*2))
            input_size*=2
        self.fc.append(torch.nn.Linear(input_size, output_size))
        #self.sigmoid = torch.sigmoid

    def forward(self, X):
        y = X
        for layer in self.fc:
            y = layer(y)
        y = torch.softmax(y, dim=1)
        return y
    
i=0
train_X = torch.tensor(mlcontext.iter_train_X[i].astype('float32').values)
min, _ = train_X.min(dim=0)
max, _ = train_X.max(dim=0)
normalized_X = (train_X - min) / (max-min + 1)

normalized_X = normalized_X

train_y = torch.tensor(mlcontext.iter_objs[i]["integer_encoded_train_y"])
random_seed = 5521
input_size = len(mlcontext.iter_train_X[i].columns)
num_classes = 5
layer_count = 4
num_epochs = 5  
one_hot_encoded = torch.nn.functional.one_hot(train_y.to(torch.int64), num_classes).float()
torch.manual_seed(random_seed)
net = SimpleNetSoftmax(input_size, num_classes, layer_count)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.005)

#Training loop
for epoch in range(num_epochs):
    outputs = net(normalized_X)
    loss = criterion(outputs, one_hot_encoded)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#Evluation     
test_X = torch.tensor(mlcontext.iter_test_X[i].astype('float32').values)
min, _ = test_X.min(dim=0)
max, _ = test_X.max(dim=0)
test_X_n = (test_X - min) / (max-min + 1)

test_X_n = test_X_n

pred = net(test_X_n)
_ , pred = torch.max(pred, 1) #Reverse One Hot
#pred_mem = pred.tolist()
import pandas as pd
from alibi.confidence import TrustScore
import numpy as np
def get_trust_scores(train_X, train_y, test_X, test_pred):  
    #test_pred = pd.DataFrame(test_pred)  
    #class_mapping = {"low": 0, "low-med": 1, "medium": 2, "med-high": 3, "high": 4}
    #train_y= train_y["Risk Level"].map(class_mapping)
    #test_pred = test_pred[0].map(class_mapping)
    ts = TrustScore()
    ts.fit(train_X, train_y, classes=5)
    #classes = {"low", "low-med", "medium", "med-high", "high"}
    score, closest_class = ts.score(test_X, test_pred, k=2)
    return score


#df_ = pd.DataFrame(train_y.numpy().reshape(-1,1))
#df_ = df_.rename(columns={0: "Risk Level"})
t = get_trust_scores(normalized_X.numpy(), pd.DataFrame(train_y.numpy().reshape(-1,1).astype(dtype = np.int64))[0], test_X_n.numpy(), pred.numpy().reshape(-1,1))
print(t)

In [ ]:
len(t)

In [ ]:
from alibi.confidence import TrustScore
def get_trust_scores(train_X, train_y, test_X, test_pred):    
    ts = TrustScore()
    ts.fit(train_X, train_y, classes=5)
    #classes = {"low", "low-med", "medium", "med-high", "high"}
    score, closest_class = ts.score(test_X, test_pred, k=2)
    return score
ts = get_trust_scores(train_X.numpy(), train_y.numpy(), test_X_n.numpy(), pred)

In [ ]:
i=0
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded_train_y = label_encoder.fit_transform(mlcontext.iter_train_y[i].values.ravel())
integer_encoded_test = label_encoder.transform(mlcontext.iter_test_y[i].values.ravel())

In [ ]:
integer_encoded_test

In [ ]:
mlcontext.iter_train_y[0].columns

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from NextVisionML.util import update_object_attributes
from alibi.confidence import TrustScore
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

def get_trust_scores(train_X, train_y, test_X, test_pred):  
    test_pred = pd.DataFrame(test_pred)  
    class_mapping = {"low": 0, "low-med": 1, "medium": 2, "med-high": 3, "high": 4}
    train_y= train_y["Risk Level"].map(class_mapping)
    test_pred = test_pred[0].map(class_mapping)
    ts = TrustScore()
    ts.fit(train_X, train_y, classes=5)
    #classes = {"low", "low-med", "medium", "med-high", "high"}
    score, closest_class = ts.score(test_X, test_pred, k=2)
    return score
i=0
dtc = DecisionTreeClassifier(
    max_depth = 3,
    min_samples_leaf = 2,
    random_state = 5,
    max_features = 3
)
dtc.fit(mlcontext.iter_train_X[i].values, mlcontext.iter_train_y[i].values)
#self.mlContext.iter_objs[i]["model"]["dtc"] = dtc
eval_predict = dtc.predict(mlcontext.iter_test_X[i].values) 
balanced_accuracy = balanced_accuracy_score(mlcontext.iter_test_y[i], eval_predict) 

mlcontext.iter_objs[i]["dtc_pred"] = eval_predict
mlcontext.iter_objs[i]["dtc_balanced_accuracy"] = balanced_accuracy
mlcontext.iter_objs[i]["dtc_trust_scores"] = get_trust_scores(mlcontext.iter_train_X[i].values, mlcontext.iter_train_y[i], mlcontext.iter_test_X[i].values, eval_predict.reshape(-1, 1))

In [ ]:
len(mlcontext.iter_objs[i]["dtc_trust_scores"])

In [ ]:
mlcontext.iter_train_y[i]["Risk Level"].unique()

In [ ]:
mlcontext.iter_train_y[i].shape

In [ ]:
import pandas as pd
from sqlalchemy import MetaData, select, text
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, registry
from NextVisionML.util import get_engine 
engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)    
mapper_registry = registry()

sqlContext = dict()
sqlContext["metadata"] = metadata
sqlContext["Base"] = Base
sqlContext["engine"] = engine    

#Metadata
datapoint = metadata.tables['datapoint']
datapoint_feature_value = metadata.tables['datapoint_feature_value']
feature = metadata.tables['feature']
datapoint_mappings = metadata.tables['datapoint_mappings']
datapoint_class_label = metadata.tables["datapoint_class_label"]
datapoint_rul_label = metadata.tables["datapoint_rul_label"]
label = metadata.tables["label"]    

# For train set
query_train = select(
    datapoint.c.id.label('datapoint_id'),
    feature.c.name.label('feature_name'),
    datapoint_feature_value.c.value.label('feature_value')
).select_from(
    datapoint.join(datapoint_feature_value, datapoint.c.id == datapoint_feature_value.c.datapoint_id)
    .join(feature, feature.c.id == datapoint_feature_value.c.feature_id)
    .join(datapoint_mappings, datapoint.c.datapoint_mappings_id == datapoint_mappings.c.id)
).where(
    datapoint_mappings.c.grouping == 'train'
)

# For test set
query_test = select(
    datapoint.c.id.label('datapoint_id'),
    feature.c.name.label('feature_name'),
    datapoint_feature_value.c.value.label('feature_value')
).select_from(
    datapoint.join(datapoint_feature_value, datapoint.c.id == datapoint_feature_value.c.datapoint_id)
    .join(feature, feature.c.id == datapoint_feature_value.c.feature_id)
    .join(datapoint_mappings, datapoint.c.datapoint_mappings_id == datapoint_mappings.c.id)
).where(
    datapoint_mappings.c.grouping == 'test'
)

# Construct SQL query for categorical labels
query_cat_labels = select(
    datapoint.c.id.label('datapoint_id'),
    label.c.name.label('label_name'),
    datapoint_class_label.c.value.label('label_value')
).select_from(
    datapoint.join(datapoint_class_label, datapoint.c.id == datapoint_class_label.c.datapoint_id)
    .join(label, label.c.id == datapoint_class_label.c.label_id)
)

# Construct SQL query for continuous labels
query_cont_labels = select(
    datapoint.c.id.label('datapoint_id'),
    label.c.name.label('label_name'),
    datapoint_rul_label.c.value.label('label_value')
).select_from(
    datapoint.join(datapoint_rul_label, datapoint.c.id == datapoint_rul_label.c.datapoint_id)
    .join(label, label.c.id == datapoint_rul_label.c.label_id)
)

con = engine.connect()
    
    


    

In [ ]:
select_statement = query_train
test = list()
with engine.connect() as conn:
    with conn.execution_options(yield_per=10000).execute(
        select_statement
    ) as result:
        for partition in result.partitions():
            # partition is an iterable that will be at most 100 items
            for row in partition:
                test.append(row)
pd.DataFrame(test)

In [ ]:
result = con.execute(query_train)
metadata.tables['datapoint']
rows = result.fetchall()

In [ ]:
import pandas as pd
from sqlalchemy import MetaData, select, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, registry
from NextVisionML.util import get_engine 
from concurrent.futures import ThreadPoolExecutor
from concurrent import futures

engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)    
mapper_registry = registry()
con = engine.connect()

def read_table_rows(table, offset, limit, i):
    print("Start " + str(i))
    with Session(engine) as session:
        # Build the SELECT statement with OFFSET and LIMIT
        re = session.query(table).order_by(table.c.id).offset(offset).limit(limit).all()
        print("End " + str(i))
        return re
    

def get_df_from_table(table):
    num_threads = 48
    table = metadata.tables[table]
    count_statement = select(func.count().label('count')).select_from(table)
    total_rows = con.execute(count_statement).scalar()
    chunk_size = total_rows // num_threads
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures_l = []

        # Submit tasks for each chunk
        for i in range(num_threads):
            offset = i * chunk_size
            limit = chunk_size if i < num_threads - 1 else total_rows - offset
            future = executor.submit(read_table_rows, table, offset, limit, i)
            futures_l.append(future)
        # Wait for all tasks to complete
        futures.wait(futures_l)
    all_rows = []
    for future in futures_l:
        all_rows.extend(future.result())
    return pd.DataFrame(all_rows)


#datapoint = get_df_from_table("datapoint")
datapoint_feature_value =get_df_from_table("datapoint_feature_value")
#feature = get_df_from_table("feature")
#datapoint_mappings =get_df_from_table("datapoint_mappings")#
#datapoint_class_label =get_df_from_table("datapoint_class_label")
#datapoint_rul_label = get_df_from_table("datapoint_rul_label")
#label = get_df_from_table("label")
datapoint.describe()

In [ ]:
import pandas as pd
from sqlalchemy import MetaData, select, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, registry
from NextVisionML.util import get_engine 
from concurrent.futures import ThreadPoolExecutor
from concurrent import futures
import copy

engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)    
mapper_registry = registry()
con = engine.connect()

def read_table_rows(table, offset, limit, i):
    select_statement = select(table).order_by(table.c.id).offset(offset).limit(limit)
    test = list()
    with engine.connect() as conn:
        with conn.execution_options(yield_per=10000, stream_results=True).execute(
            select_statement
        ) as result:
            for i, partition in enumerate(result.partitions()):
                if(i==1): 
                    print(str(offset) + " " + str(limit) + " "+ str(i), end='\r')
                for row in partition:
                    test.append(row)
    return test
    

def get_df_from_table(table):
    num_threads = 48
    table = metadata.tables[table]
    count_statement = select(func.count().label('count')).select_from(table)
    total_rows = con.execute(count_statement).scalar()
    chunk_size = total_rows // num_threads
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures_l = []

        # Submit tasks for each chunk
        for i in range(num_threads):
            offset = i * chunk_size
            limit = chunk_size if i < num_threads - 1 else total_rows - offset
            future = executor.submit(read_table_rows, table, copy.deepcopy(offset), copy.deepcopy(limit), i)
            futures_l.append(future)
        # Wait for all tasks to complete
        futures.wait(futures_l)
    all_rows = []
    for future in futures_l:
        all_rows.extend(future.result())
    return pd.DataFrame(all_rows)

datapoint_feature_value =get_df_from_table("datapoint_feature_value")
datapoint_feature_value.describe()

In [ ]:
import pandas as pd
from sqlalchemy import MetaData, select, func, text
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, registry
from NextVisionML.util import get_engine 
from concurrent.futures import ThreadPoolExecutor
from concurrent import futures

engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)    
mapper_registry = registry()
con = engine.connect()
from sqlalchemy import text


In [ ]:
def get_df_from_table(table_name):    
    test = list()
    with engine.connect() as conn:
        with conn.execution_options(yield_per=100000).execute(
            text("SELECT * FROM " + table_name)
        ) as result:
            for partition in result.partitions():
                # partition is an iterable that will be at most 100 items
                for row in partition:
                    test.append(row)
    return pd.DataFrame(test)

datapoint_feature_value =get_df_from_table("datapoint_feature_value")

In [ ]:
session = Session(engine)

test = session.query(metadata.tables["datapoint_feature_value"]).all()


In [ ]:
session = Session(engine)
table = metadata.tables["datapoint_feature_value"]
test = session.query(metadata.tables["datapoint_feature_value"]).order_by(table.c.id).offset(1000).limit(10000).all()
print(d)

In [ ]:


# Number of concurrent threads to use
 # Adjust this based on your system's capabilities

# Determine the total number of rows in the table
with engine.connect() as connection:
    count_statement = select([table]).count()
    total_rows = connection.execute(count_statement).scalar()

# Calculate the chunk size for each thread
chunk_size = total_rows // num_threads

# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = []

    # Submit tasks for each chunk
    for i in range(num_threads):
        offset = i * chunk_size
        limit = chunk_size if i < num_threads - 1 else total_rows - offset
        future = executor.submit(read_table_rows, offset, limit)
        futures.append(future)

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

# Retrieve results from completed futures
all_rows = []
for future in futures:
    all_rows.extend(future.result())

In [ ]:
df_train = pd.read_sql_query(query_train, con)
    df_test = pd.read_sql_query(query_test, con)

    # Pivot tables to get features as columns, for both train and test
    train = df_train.pivot_table(index='datapoint_id', columns='feature_name', values='feature_value').reset_index()
    test = df_test.pivot_table(index='datapoint_id', columns='feature_name', values='feature_value').reset_index()
    
    # Execute the queries and load into DataFrames
    df_cat_labels = pd.read_sql_query(query_cat_labels, con)
    df_cont_labels = pd.read_sql_query(query_cont_labels, con)

    # Pivot table for categorical labels
    pivot_cat_labels = df_cat_labels.pivot_table(
        index='datapoint_id',
        columns='label_name',
        values='label_value',
        aggfunc='first'  # Use 'first' or 'max' for categorical data
    ).reset_index()

    # Pivot table for continuous labels
    pivot_cont_labels = df_cont_labels.pivot_table(
        index='datapoint_id',
        columns='label_name',
        values='label_value',
        aggfunc='mean'  # Use 'mean' or another suitable function for numerical data
    ).reset_index()
    
    # Merge categorical and continuous labels
    pivot_labels = pd.merge(pivot_cat_labels, pivot_cont_labels, on='datapoint_id', how='outer')

    # Merge labels with feature dataframes
    train_with_labels = pd.merge(train, pivot_labels, on='datapoint_id', how='left')
    test_with_labels = pd.merge(test, pivot_labels, on='datapoint_id', how='left')

    return sqlContext, train_with_labels, test_with_labels
    